In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# import plotting
import scipy.stats as ss

# import machine learning
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler, SMOTE
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, cohen_kappa_score
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:

injury = pd.read_csv("../input/nfl-playing-surface-analytics/InjuryRecord.csv")
play = pd.read_csv("../input/nfl-playing-surface-analytics/PlayList.csv")
track = pd.read_csv("../input/nfl-playing-surface-analytics/PlayerTrackData.csv")

Injury Record: The injury record file in .csv format contains information on 105 lower-limb injuries that occurred during regular season games over the two seasons. Injuries can be linked to specific records in a player history using the PlayerKey, GameID, and PlayKey fields.

['PlayerKey', 'GameID', 'PlayKey', 'BodyPart', 'Surface', 'DM_M1',
     'DM_M7', 'DM_M28', 'DM_M42']
Play List: – The play list file contains the details for the 267,005 player-plays that make up the dataset. Each play is indexed by PlayerKey, GameID, and PlayKey fields. Details about the game and play include the player’s assigned roster position, stadium type, field type, weather, play type, position for the play, and position group.

['PlayerKey', 'GameID', 'PlayKey', 'RosterPosition', 'PlayerDay',
     'PlayerGame', 'StadiumType', 'FieldType', 'Temperature', 'Weather',
     'PlayType', 'PlayerGamePlay', 'Position', 'PositionGroup']
Player Track Data: player level data that describes the location, orientation, speed, and direction of each player during a play recorded at 10 Hz (i.e. 10 observations recorded per second).
['PlayKey', 'time', 'event', 'x', 'y', 'dir', 'dis', 'o', 's']

In [ ]:
injury.head()

In [ ]:
sns.catplot(x="BodyPart", data=injury, kind="count")


In [ ]:
sns.catplot(x="BodyPart", data=injury,hue='Surface', kind="count")


In [ ]:
sns.countplot(x="Surface", data=injury)

In [ ]:
injury['injury_type']=injury[['DM_M1','DM_M7','DM_M28','DM_M42']].sum(axis=1)
def change_vals(i):
    if i==1:
        return 'Light Injury'
    elif i==2:
        return 'Medium Injury'
    elif i==3:
        return 'Almost Serious Injury'
    else:
        return 'Serious Injury'
injury['injury_type']=injury['injury_type'].apply(change_vals)

In [ ]:
sns.catplot(x="Surface", data=injury,hue='BodyPart', kind="count")


In [ ]:
sns.catplot(x="Surface", data=injury,hue='injury_type', kind="count")


In [ ]:
body_part=injury['BodyPart'].unique().tolist()
fig=plt.figure(figsize=(15,3))
for i in range(len(body_part)-1):
    ax=fig.add_subplot(1,5,i+1)
    c=injury[(injury['BodyPart']==body_part[i])&(injury['Surface']=='Synthetic')]['injury_type'].value_counts(normalize=True)*100
    c_lst=c.tolist()
    ax.bar(c.index,c_lst,width=0.4)
    ax.set_title('{} in Synthetic'.format(body_part[i]))
    ax.set_ylabel('')
    xlabels=[i for i in c.index]
    ax.set_xticklabels(xlabels, rotation=90)
plt.show()

In [ ]:
injury.BodyPart.value_counts()


In [ ]:
injury.DM_M1.value_counts()


In [ ]:
sns.catplot(x="DM_M1", hue='BodyPart', data=injury, kind="count")


In [ ]:
injury.DM_M7.value_counts()


In [ ]:
sns.catplot(x="DM_M7", hue='BodyPart', data=injury[(injury.DM_M7==1)], kind="count")

In [ ]:
#                 how='left')
Injury_games_play = injury.merge(play,
                  on='PlayKey',
                  how='left')

In [ ]:
Injury_games_play.head()


In [ ]:
Injury_games_play['Weather'].unique()

In [ ]:
Injury_games_play.isnull().sum()


In [ ]:
Injury_games_play = Injury_games_play[pd.notnull(Injury_games_play['PlayKey'])]


In [ ]:
Injury_games_play.tail()


In [ ]:
Injury_games_play.columns


In [ ]:
sns.catplot(x="BodyPart", hue='StadiumType', data=Injury_games_play, kind="count")


In [ ]:
Injury_games_play=Injury_games_play.dropna()


In [ ]:
data=Injury_games_play[['BodyPart', 'Surface', 'RosterPosition', 'PlayerDay', 'PlayerGame', 'StadiumType', 'FieldType',
       'Temperature', 'Weather', 'PlayType', 'PlayerGamePlay', 'Position',
       'PositionGroup']]

ax = sns.pairplot(data, hue='BodyPart', kind="reg")

**now for player data**

In [ ]:
chart=sns.catplot(x="RosterPosition", data=play, kind="count")
chart.set_xticklabels(rotation=45)

plt.gcf().set_size_inches(16, 8)


In [ ]:
chart=sns.catplot(x="PlayType", data=play, kind="count")

chart.set_xticklabels(rotation=45)

plt.gcf().set_size_inches(16, 8)

In [ ]:
injury_games_play = injury.merge(play,
                  on='PlayKey',
                  how='left')

In [ ]:
injury_games_play.head()

In [ ]:
injury_games_play['Weather'].unique()

In [ ]:
injury_games_play.shape

In [ ]:
injury_games_play['Weather'].value_counts()

In [ ]:

# def merge(data):
#     cold=0
#     warm=0
#     for i in   injury_games_play['Weather'].tolist():
#         if 'cloud' in i:
#             cold+=1
#         else:
#             warm+=1
# print(merge(injury_games_play))
injury_games_play['Weather'].fillna('Cloudy',inplace=True)

In [ ]:
# def merge(data):
#     for i in data.Weather.values:
#         if 'Cloud' in ''.join(i):
#             data['weather_type']='cold'
#         else:
#             data['weather_type']='warm'
# merge(injury_games_play)  
# injury_games_play['weather_type'].value_counts()
# injury_games_play.Weather.values
from collections import Counter
cold=0;warm=0
for i in list(Counter(injury_games_play.Weather.values).keys()):
    if 'cloud' in i.replace('','').lower():
        cold+=Counter(injury_games_play.Weather.values)[i]
    else:
        warm+=Counter(injury_games_play.Weather.values)[i]
cold,warm        
    
    

players tend to get hurt 61 times whenevr word 'cloud' is mentioned in weather and 44 times when its not . so this observation agrees with the scientific fact that the colder or cloudy the weather the more muscles are in contracted state and hence the risk of getting hurt is comparatively high.

In [ ]:
sns.catplot(x="BodyPart", hue='Weather', data=injury_games_play, kind="count")


definitely higher injuries in cold weather 

In [ ]:
chart = sns.catplot(x="injury_type", hue='Weather', data=injury_games_play, kind="count")

chart.set_xticklabels(rotation=45)

plt.gcf().set_size_inches(16, 8)

more evidence towards higher risk of injury in cold weather ..

In [ ]:
chart = sns.catplot(x="injury_type", hue='Surface', data=injury_games_play, kind="count")

chart.set_xticklabels(rotation=45)

plt.gcf().set_size_inches(16, 8)

more serious injuries on synthetic turf . now lets examine the position data 

In [ ]:
injury_games_play.Position.value_counts()

according to wikipeida following are the position in the football :-
Offensive (interior) line

The offensive line (on left, in orange jerseys) consists of a center (with ball in hand ready to snap) with two guards on either side, and two tackles.
The offensive line is primarily responsible for blocking. During normal play, offensive linemen do not handle the ball (aside from the snap from center), unless the ball is fumbled by a ball carrier, a pass is deflected, or when a player who is normally an offensive lineman takes a different position on the field. The offensive line consists of:
**
Center (C)**
The center is the player who begins the play from scrimmage by snapping the ball to the quarterback. As the name suggests, the center usually plays in the middle of the offensive line, though some teams may employ an unbalanced line where the center is offset to one side. Like all offensive linemen, the center has the responsibility to block defensive players. The center often also has the responsibility to call out blocking assignments and make last second adjustments depending on the defensive alignment.
**Offensive guard (OG)**
Two guards line up directly on either side of the center. Like all interior linemen, their function is to block on both running and passing plays. On some plays, rather than blocking straight ahead, a guard will "pull", whereby the guard comes out of their position in line to lead block for a ball carrier, on plays known as "traps" (for inside runs), "sweeps" (for outside runs), and "screens" (for passing plays). In such cases, the guard is referred to as a "pulling guard".

The six backs and receivers are those that line up outside or behind the offensive line. There are four main positions in this set of players:

**Wide receiver (WR)**

A wide receiver (No. 87, in white) begins a play in the flanker position
Wide receivers are pass-catching specialists. Their main job is to run pass routes and get open for passes, although they are occasionally called on to block. Wide receivers generally line up split "wide" near the sidelines at the start of the play. Wide receivers, like running backs, come in different varieties depending on exactly where they line up. A wide receiver who is directly on the line of scrimmage is called a "split end", and is counted among the seven required players on the line of scrimmage. A wide receiver who lines up behind the line (and thus counts as one of the four backs) is called the "flanker". A wide receiver who lines up between the outermost wide receiver and the offensive line is said to be "in the slot" and is called the "slot receiver".

**Tight end (TE)**
Tight ends play on either side of, and directly next to, the tackles. Tight ends are considered hybrid players, something between a wide receiver and an offensive lineman. Because they play next to the other offensive linemen, they are frequently called on to block, especially on running plays. However, because they are eligible receivers, they may also catch passes. The position known as the "H-back" is a tight end who lines up behind the line of scrimmage, and is thus counted as one of the four "backs", but otherwise their role is similar to that of other tight ends.
Depending on the style of offense the coaches have designed, the game situation, and the relative skill sets of the players, teams may run formations which contain any number of running backs, wide receivers, and tight ends, so long as the mandated "four backs and seven on the line" rule is followed. For many years, the standard set consisted of the quarterback, two running backs (a tailback/halfback and a fullback), two wide receivers (a flanker and a split end) and a tight end. Modern teams show a wide variety of formations, from a "full house" formation with three running backs, two tight ends, and no wide receivers, to "spread" formations featuring four or five wide receivers, sometimes without any running backs. The "I formation" is one of the most common.

Defense
The defensive team or defense is the team that begins a play from scrimmage not in possession of the ball. The objective of the defensive team is to prevent the other team from scoring. The defense accomplishes this by forcing the offense to turn the ball over, either by preventing them from achieving a first down and forcing a punt, forcing the offense to fumble or throw an interception, or more rarely, forcing a turnover on downs.

Unlike the offensive team, the rules do not restrict the defensive team into certain positions. A defensive player may line up anywhere on his side of the line of scrimmage and perform any legal action. Over time, however, defensive roles have become defined into three main sets of players, and several individual positions.

Defensive line

The four defensive linemen (in red) have their hands on the ground in a "three point stance"
Like their offensive counterparts, defensive linemen (also called rushers) line up directly on the line of scrimmage, close to the ball. There are two positions usually considered part of the defensive line:

**Defensive tackle (DT);**
Sometimes called a "defensive guard", defensive tackles play at the center of the defensive line. Their function is to rush the passer and stop running plays directed at the middle of the line of scrimmage. The most interior defensive tackle who sometimes lines up directly across from the ball (and therefore is almost nose-to-nose with the offense's center) is often called a "nose tackle", alternately "nose guard" or "middle guard". The nose tackle is most common in the 3-4 defense. Most defensive sets have one or two defensive tackles. If one employs a second defensive tackle, sometimes called an "under tackle", they are usually a bit faster than the nose tackle.
Defensive end (DE)
The two defensive ends play next to the defensive tackles, at the edges of the defensive line. Their function is to attack the passer or stop offensive runs to the outer edges of the line of scrimmage (most often referred to as "containment"). The faster of the two is usually placed on the right side of the defensive line (quarterback's left) because that is a right-handed quarterback's blind side.
Often, a defensive lineman will have their hands on the ground, in a three- or four-point stance before the ball is snapped; this distinguishes their pre-snap stance from a linebacker, who begins in a two-point stance (i.e. without a hand touching the ground).

Linebackers

This defense (in white) is in a base 4-3 set. Just behind the four defensive linemen (whose hands are on the ground) are three linebackers (Nos. 55, 3 and 16), and further back are two safeties (numbers 24 and 44).
Linebackers play behind the defensive line and perform various duties depending on the situation, including rushing the passer, covering receivers, and defending against the run.

**Middle linebacker (MLB)**
Sometimes called the "inside linebacker" (especially in a 3-4 defense), and known colloquially as the "Mike" linebacker, the middle linebacker is often known as the "quarterback of the defense", as they are frequently the primary defensive play callers and must react to a wide variety of situations. Middle linebackers must be capable of stopping running backs who make it past the defensive line, covering pass plays over the middle, and rushing the quarterback on blitz plays.
**Outside linebacker (OLB)**
Outside linebackers are given different names depending on their role and the philosophy of the team. Some teams keep their outside linebackers on the same side of the field at all times. Some teams define them by their role; as playing either "strongside" (SLB) or "weakside" (WLB). The strongside, or "Sam", linebacker lines up on the same side as the offensive tight end and often is responsible for covering the tight end or running back on pass plays. The weakside, or "Will", linebacker lines up on the side of the offensive line without a tight end, and is often used to rush, or blitz the quarterback, or may need to cover a running back on pass plays.
Defensive backs

Cornerback Tory James gets a read on the offense just prior to the start of play
Defensive backs, also known as the "secondary", play either behind the linebackers or set to the outside, near the sidelines. Defensive backs are primarily used to defend against pass plays. Defensive backs also act as the last line of defense on running plays and need to be able to make open field tackles, especially when the ball carrier has gotten past the other defenders. A normal complement of defensive backs includes two cornerbacks and two safeties, though specialty defensive backs (nickelbacks and dime backs) can be brought in in place of linebackers and defensive linemen, when there is a need to cover additional pass receivers.

**Cornerback (CB)**
Cornerbacks attempt to prevent successful passes by either swatting the airborne ball away from the receiver or by catching the pass themselves. In rushing situations, their job is to contain the runner, either by directing them back to the middle of the field to be tackled, by tackling them themselves or by forcing them out of bounds.
Safety (S)
The safeties are the last line of defense (farthest from the line of scrimmage) and usually help the corners with deep-pass coverage. The strong safety (SS) is usually the larger and stronger of the two, providing extra protection against run plays by standing closer to the line of scrimmage, usually on the strong (tight end) side of the field. The free safety (FS) is usually the smaller and faster of the two, and is usually the deepest player on the defense, providing help on long pass plays.
Nickelback and dimeback
In certain formations, the defense may remove a linebacker or a defensive lineman to bring in extra pass coverage in the form of extra defensive backs. A formation with five defensive backs is often called a "nickel" formation, and the fifth (extra) defensive back is called a "nickelback" after the U.S. nickel coin, a five-cent piece. By extension, a formation with a sixth defensive back (dimeback) is called a "dime package", a 10-cent dime coin being two nickels (nickelbacks). Rarely, a team may employ seven or eight defensive backs on certain plays.
Defensive formations are often known by a numerical code indicating the number of players at each position. The two most common formations are the 3–4 defense and the 4–3 defense, where the first number refers to the number of defensive linemen, and the second number refers to the number of linebackers (the number of defensive backs can be inferred, since there must be eleven players on the field). Thus, a 3–4 defense consists of three defensive linemen (usually a nose tackle and two defensive ends), four linebackers, and four defensive backs (two cornerbacks, a strong safety, and a free safety)

**players who have more to do with the ball passing and catching have a higher chance of getting hurt than the players who's job is to defend . interesting ! **

In [ ]:
chart = sns.catplot(x="Position", hue='Surface', data=injury_games_play, kind="count")

chart.set_xticklabels(rotation=45)

plt.gcf().set_size_inches(16, 8)

In [ ]:
chart = sns.catplot(x="Position", hue='injury_type', data=injury_games_play, kind="count")

chart.set_xticklabels(rotation=45)

plt.gcf().set_size_inches(16, 8)

In [ ]:
injury_games_play = Injury_games_play[pd.notnull(Injury_games_play['PlayKey'])]
sns.catplot(x="BodyPart", hue='StadiumType', data=Injury_games_play, kind="count")


In [ ]:
injury_games_play.columns

In [ ]:
injury_games_play.head()

now lets try to examine the PlayType and its impact on the injury..

In [ ]:
injury_games_play.PlayType.unique()

In [ ]:
injury_games_play.PlayType.value_counts()

In [ ]:
chart = sns.catplot(x="PlayType", hue='injury_type', data=injury_games_play, kind="count")

chart.set_xticklabels(rotation=45)

plt.gcf().set_size_inches(16, 8)

seems like players are most likely to get injured during passing and rush . interersting ..

In [ ]:
chart = sns.catplot(x="PlayType", hue='BodyPart', data=injury_games_play, kind="count")

chart.set_xticklabels(rotation=45)

plt.gcf().set_size_inches(16, 8)

right off the bat we can see that the chance for the foot injury is really less . But while doing punt,pass n kicoff the knee is most likely to get injured only in the case of rush the ankle injury is high .

In [ ]:
chart = sns.catplot(x="DM_M1", hue='injury_type', data=injury_games_play, kind="count")

chart.set_xticklabels(rotation=45)

plt.gcf().set_size_inches(16, 8)

we can see that when player comes back after one holiday the chances for the medium injury is the highest , this is intuitive since longer the ring rust more likely it is that the player gets injured .

In [ ]:
chart = sns.catplot(x="DM_M7", hue='injury_type', data=injury_games_play, kind="count")

chart.set_xticklabels(rotation=45)

plt.gcf().set_size_inches(16, 8)

In [ ]:
chart = sns.catplot(x="DM_M28", hue='injury_type', data=injury_games_play, kind="count")

chart.set_xticklabels(rotation=45)

plt.gcf().set_size_inches(16, 8)

In [ ]:
chart = sns.catplot(x="DM_M42", hue='injury_type', data=injury_games_play, kind="count")

chart.set_xticklabels(rotation=45)

plt.gcf().set_size_inches(16, 8)

in all the cases when the players absense was more than 7 days the chance of serious injury has increased significantly ..

In [ ]:
data=injury_games_play[['BodyPart', 'Surface', 'RosterPosition', 'PlayerDay', 'PlayerGame', 'StadiumType', 'FieldType',
       'Temperature', 'Weather', 'PlayType', 'PlayerGamePlay', 'Position',
       'PositionGroup']]

ax = sns.pairplot(data, hue='BodyPart', kind="reg")

In [ ]:
sns.heatmap(injury.corr())

In [ ]:
sns.heatmap(injury_games_play.corr())

In [ ]:
track.head()

In [ ]:
track['event'].value_counts()

In [ ]:
# injury_games_play_track = injury_games_play.merge(track,
#                   on='PlayKey',
#                   how='left')

In [ ]:
# create a dataframe with game-level information
game_df = play[['GameID', 'StadiumType', 'FieldType', 'Weather', 'Temperature']].drop_duplicates().reset_index().drop(columns=['index'])

In [ ]:
def clean_weather(row):
    cloudy = ['Cloudy 50% change of rain', 'Hazy', 'Cloudy.', 'Overcast', 'Mostly Cloudy',
          'Cloudy, fog started developing in 2nd quarter', 'Partly Cloudy',
          'Mostly cloudy', 'Rain Chance 40%',' Partly cloudy', 'Party Cloudy',
          'Rain likely, temps in low 40s', 'Partly Clouidy', 'Cloudy, 50% change of rain','Mostly Coudy', '10% Chance of Rain',
          'Cloudy, chance of rain', '30% Chance of Rain', 'Cloudy, light snow accumulating 1-3"',
          'cloudy', 'Coudy', 'Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.',
         'Cloudy fog started developing in 2nd quarter', 'Cloudy light snow accumulating 1-3"',
         'Cloudywith periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.',
         'Cloudy 50% change of rain', 'Cloudy and cold',
       'Cloudy and Cool', 'Partly cloudy']
    
    clear = ['Clear, Windy',' Clear to Cloudy', 'Clear, highs to upper 80s',
             'Clear and clear','Partly sunny',
             'Clear, Windy', 'Clear skies', 'Sunny', 'Partly Sunny', 'Mostly Sunny', 'Clear Skies',
             'Sunny Skies', 'Partly clear', 'Fair', 'Sunny, highs to upper 80s', 'Sun & clouds', 'Mostly sunny','Sunny, Windy',
             'Mostly Sunny Skies', 'Clear and Sunny', 'Clear and sunny','Clear to Partly Cloudy', 'Clear Skies',
            'Clear and cold', 'Clear and warm', 'Clear and Cool', 'Sunny and cold', 'Sunny and warm', 'Sunny and clear']
    
    rainy = ['Rainy', 'Scattered Showers', 'Showers', 'Cloudy Rain', 'Light Rain', 'Rain shower', 'Rain likely, temps in low 40s.', 'Cloudy, Rain']
    
    snow = ['Heavy lake effect snow']
    
    indoor = ['Controlled Climate', 'Indoors', 'N/A Indoor', 'N/A (Indoors)']
        
    if row.Weather in cloudy:
        return 'Cloudy'
    
    if row.Weather in indoor:
        return 'Indoor'
    
    if row.Weather in clear:
        return 'Clear'
    
    if row.Weather in rainy:
        return 'Rain'
    
    if row.Weather in snow:
        return 'Snow'
      
    if row.Weather in ['Cloudy.', 'Heat Index 95', 'Cold']:
        return np.nan
    
    return row.Weather

def clean_stadiumtype(row):
    if row.StadiumType in ['Bowl', 'Heinz Field', 'Cloudy']:
        return np.nan
    else:
        return row.StadiumType

def clean_play_df(play_df):
    play_df_cleaned = play_df.copy()
    
    # clean StadiumType
    play_df_cleaned['StadiumType'] = play_df_cleaned['StadiumType'].str.replace(r'Oudoor|Outdoors|Ourdoor|Outddors|Outdor|Outside', 'Outdoor')
    play_df_cleaned['StadiumType'] = play_df_cleaned['StadiumType'].str.replace(r'Indoors|Indoor, Roof Closed|Indoor, Open Roof', 'Indoor')
    play_df_cleaned['StadiumType'] = play_df_cleaned['StadiumType'].str.replace(r'Closed Dome|Domed, closed|Domed, Open|Domed, open|Dome, closed|Domed', 'Dome')
    play_df_cleaned['StadiumType'] = play_df_cleaned['StadiumType'].str.replace(r'Retr. Roof-Closed|Outdoor Retr Roof-Open|Retr. Roof - Closed|Retr. Roof-Open|Retr. Roof - Open|Retr. Roof Closed', 'Retractable Roof')
    play_df_cleaned['StadiumType'] = play_df_cleaned.apply(lambda row: clean_stadiumtype(row), axis=1)
    
    # clean Weather
    play_df_cleaned['Weather'] = play_df_cleaned.apply(lambda row: clean_weather(row), axis=1)
    
    return play_df_cleaned

In [ ]:
play_df_cleaned = clean_play_df(play)
game_df_cleaned = play_df_cleaned[['GameID', 'StadiumType', 'FieldType', 'Weather', 'Temperature']].drop_duplicates().reset_index().drop(columns=['index'])

In [ ]:
player_data_df = play_df_cleaned[['PlayerKey', 'RosterPosition', 'PlayerGamePlay', 'Position', 'PositionGroup']]

In [ ]:
ax=sns.countplot(x="RosterPosition", data=player_data_df)
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)

plt.gcf().set_size_inches(16, 8)

In [ ]:
ax=sns.countplot(x="PositionGroup", data=player_data_df)
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)

plt.gcf().set_size_inches(16, 8)

In [ ]:
ax=sns.countplot(x="Position", data=player_data_df)
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)

plt.gcf().set_size_inches(16, 8)

In [ ]:
ax=sns.countplot(x="PlayerGamePlay", data=player_data_df)
ax.set_xticklabels(ax.get_xticklabels(),rotation=60)

plt.gcf().set_size_inches(16, 10)

In [ ]:
def visualize_play(play_df_cleaned):
    plt.style.use('ggplot')
    fig, ax = plt.subplots(1,1,figsize=(15,5))
    
    plt.bar(play_df_cleaned.PlayType.value_counts().keys().values, play_df_cleaned.PlayType.value_counts().values)
    plt.xticks(range(len(play_df_cleaned.PlayType.value_counts().keys().values)), play_df_cleaned.PlayType.value_counts().keys().values, rotation=20)
    add_value_labels(ax, spacing=5)
    plt.title('Play-Level Exploration: PlayType', fontsize=16)
    
    plt.show()

In [ ]:
visualize_play(play_df_cleaned)


In [ ]:
def visualize_field_heatmap(player_df, xbins=13, ybins=6, annotate = False):
    # create a grid
    x = np.linspace(0, 120, xbins)
    y = np.linspace(0, 53, ybins)
    
    # initialize heatmap
    hmap = np.zeros((xbins,ybins))
    
    for i in range(xbins-1):
        for j in range(ybins-1):
            hmap[i,j] = len(player_df[(player_df.x >= x[i]) & (player_df.x <= x[i+1]) & (player_df.y >= y[j]) & (player_df.y <= y[j+1])])
            
    fig = plt.figure(figsize=(10,5))
    ax = sns.heatmap(np.transpose(hmap), annot=annotate, fmt = '.0f')
    plt.title('Field Heatmap \n the most visited areas of the field are highlighted')
    plt.show()

In [ ]:
visualize_field_heatmap(track)


In [ ]:
# joined cleaned games dataset and injury dataset
game_injury_df = injury.set_index('GameID').join(game_df_cleaned.set_index('GameID'), how = 'outer')

# fill null values for the injury columns with zeros
game_injury_df['DM_M1'] = game_injury_df['DM_M1'].fillna(0).astype(int)
game_injury_df['DM_M7'] = game_injury_df['DM_M7'].fillna(0).astype(int)
game_injury_df['DM_M28'] = game_injury_df['DM_M28'].fillna(0).astype(int)
game_injury_df['DM_M42'] = game_injury_df['DM_M42'].fillna(0).astype(int)

game_injury_df.DM_M1 = game_injury_df.DM_M1 - game_injury_df.DM_M7
game_injury_df.DM_M7 = game_injury_df.DM_M7 - game_injury_df.DM_M28
game_injury_df.DM_M28 = game_injury_df.DM_M28 - game_injury_df.DM_M42

# introduce a column with a flag indicating an injury
game_injury_df['Injury'] = game_injury_df['DM_M1'] + game_injury_df['DM_M7'] + game_injury_df['DM_M28'] + game_injury_df['DM_M42']

# drop duplicated surface column
game_injury_df = game_injury_df.drop(columns=['Surface'])

# drop play-level features just for now
game_injury_df = game_injury_df.drop(columns=['PlayerKey', 'PlayKey'])

# create dummy variables
game_injury_df_dummies = pd.get_dummies(game_injury_df, dummy_na = True, drop_first = True).drop(columns=['FieldType_nan'])

In [ ]:
corr_df = game_injury_df_dummies[['Temperature', 'StadiumType_Indoor', 'StadiumType_Open', 'StadiumType_Outdoor', 'StadiumType_Retractable Roof',
                                 'FieldType_Synthetic', 'Weather_Cloudy', 'Weather_Rain', 'Weather_Snow', 'Injury']].corr()

fig = plt.figure(figsize=(10,7))
sns.heatmap(corr_df, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True))
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.boxplot([game_injury_df[game_injury_df.Injury == 0].Temperature.values, game_injury_df[game_injury_df.Injury == 1].Temperature.values], vert = False)
plt.title('Temperature Distribution')
plt.yticks([1,2], ['No Injury', 'Injury'])
plt.xlim(0,100)
plt.xlabel('Temperature')
plt.show()

In [ ]:
def player_games_timeline(player_key, play_df, injury_df):
    '''
    Function to plot the player's timeline
    '''
    player_games = play_df[play_df.PlayerKey == player_key][['GameID', 'PlayKey', 'PlayerDay', 'PlayerGame']]
    
    # plot timeline for the player
    plt.figure(figsize=(20,5))
    plt.title('Player Games Timeline \n PlayerKey: ' + str(player_key))  
    plt.plot(player_games.PlayerDay.unique(), np.zeros(len(player_games.PlayerDay.unique())), color='#00c2c7')
    plt.scatter(player_games.PlayerDay.unique(), np.zeros(len(player_games.PlayerDay.unique())), s=100, color='#00c2c7', label='games')
    
    # add games with injury
    injured_players = injury_df.PlayerKey.unique()
    if player_key in injured_players:
        injury_games = injury_df[injury_df.PlayerKey == player_key].GameID.values
        injury_days = player_games[player_games.GameID.isin(injury_games)].PlayerDay.unique()
        
        plt.scatter(injury_days, np.zeros(len(injury_days)), s=100, color='#e01e5a', label='injury')
    
    plt.legend()
    plt.xlabel('days')
    plt.yticks([])
    plt.show()


In [ ]:
player_games_timeline(26624, play, injury)


In [ ]:
# joined cleaned games dataset and injury dataset
play_injury_df = injury.dropna(subset=['PlayKey']).set_index('PlayKey').join(play_df_cleaned.set_index('PlayKey'), how = 'outer', lsuffix='_left', rsuffix='_right')

# fill null values for the injury columns with zeros
play_injury_df['DM_M1'] = play_injury_df['DM_M1'].fillna(0).astype(int)
play_injury_df['DM_M7'] = play_injury_df['DM_M7'].fillna(0).astype(int)
play_injury_df['DM_M28'] = play_injury_df['DM_M28'].fillna(0).astype(int)
play_injury_df['DM_M42'] = play_injury_df['DM_M42'].fillna(0).astype(int)

# introduce a column with a flag indicating an injury
play_injury_df.DM_M1 = play_injury_df.DM_M1 - play_injury_df.DM_M7
play_injury_df.DM_M7 = play_injury_df.DM_M7 - play_injury_df.DM_M28
play_injury_df.DM_M28 = play_injury_df.DM_M28 - play_injury_df.DM_M42

play_injury_df['Injury'] = play_injury_df['DM_M1'] + play_injury_df['DM_M7'] + play_injury_df['DM_M28']+ play_injury_df['DM_M42']

# drop duplicated surface column
play_injury_df = play_injury_df.drop(columns=['Surface'])

# create dummy variables
play_injury_df_dummies = pd.get_dummies(play_injury_df, columns = ['PlayType', 'PositionGroup'], dummy_na = True, drop_first = True)


In [ ]:
# reset index 
features_df = play_injury_df.copy().reset_index()
# drop the irrelevant columns
features_df = features_df.drop(columns=['PlayerKey_left','GameID_left','BodyPart', 'PlayKey', 'PlayerKey_right', 'GameID_right', 'DM_M1', 'DM_M7', 'DM_M28', 'DM_M42'])
# convert into dummies
features_df = pd.get_dummies(features_df, dummy_na = False, drop_first = True)

In [ ]:
# split into X and y
y = features_df['Injury']
X = features_df.drop(columns=['Injury'])

In [ ]:
skf = StratifiedKFold(n_splits=2)

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.values[train_index, :], X.values[test_index, :]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model=RandomForestClassifier()
                      
model.fit(X_train, y_train)

since the dataset is small ,classifier will always predict no injury so accuracy won't be a good parameter to judge model performnce so we will use auc roc and then feature importances...

In [ ]:
y_prob = model.predict_proba(X_test)[:,1] # This will give you positive class prediction probabilities  
y_pred = np.where(y_prob > 0.5, 1, 0) # This will threshold the probabilities to give class predictions.
model.score(X_test, y_pred)

In [ ]:
from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

In [ ]:
# extract the feature importances from XGBoost model
feature_importances = model.feature_importances_
feature_importances = pd.DataFrame(feature_importances, index = X.columns).reset_index().rename(columns={'index':'feature', 0:'importance'}).sort_values(by=['importance'], ascending=False)


In [ ]:
# plot as a bar chart
plt.figure(figsize=(20,7))
plt.bar(range(len(feature_importances)), feature_importances.importance.values)
plt.xticks(range(len(feature_importances)), feature_importances.feature.values, rotation=90)

plt.title('Feature importances')
plt.xlabel('features')
plt.show()

Motion Analysis

In [ ]:
dfs=[track,injury_games_play]

In [ ]:
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right,on='PlayKey'), dfs)


In [ ]:
df_final.head()


In [ ]:
df_final.columns

In [ ]:
df_final.s.value_counts()

In [ ]:
ax = sns.distplot(df_final['s'])
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)

plt.gcf().set_size_inches(16, 8)

In [ ]:
df_final.groupby('PlayKey')[['s']].max() \
    .sort_values('s', ascending=False) \
    .query('s != 0').head(10) \
    .plot(kind='barh', figsize=(15, 5), title='Top 10 Plays by Max Player Speed')
plt.show()

df_final.groupby('PlayKey')[['s']].max() \
    .sort_values('s', ascending=True) \
    .query('s != 0').head(10) \
    .plot(kind='barh', figsize=(15, 5), title='Bottom 10 Plays by Min Player Speed')
plt.show()

In [ ]:
ax=sns.scatterplot(x="Temperature", y="s", data=df_final)
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)
plt.gcf().set_size_inches(16, 8)

In [ ]:
ax=sns.scatterplot(x="Temperature", y="s", hue = 'injury_type',data=df_final)
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)
plt.gcf().set_size_inches(16, 8)

In [ ]:
from scipy.stats.stats import pearsonr

ax=sns.jointplot(x="Temperature", y="s", data=df_final,stat_func=pearsonr);
# ax.set_xticklabels(ax.get_xticklabels(),rotation=30)
plt.gcf().set_size_inches(16, 8)

pearson corelation suggests there is indeed a corelation between speed and temperature

references :- https://www.kaggle.com/aleksandradeis/nfl-injury-analysis,
                https://www.kaggle.com/chandraroy/nfl-analytics